In [77]:
import os

import pandas as pd

if not os.path.basename(os.getcwd()) == "ThousandSuns":
    os.chdir("..")  # Run Once!
%load_ext autoreload
%autoreload 2

from output_graphs.hourly_graph_creator import *
from df_objects.df_objects import ElectricityUseDf, DemandDf, ProductionDf, CostElectricityDf
from hourly_simulation import strategies
from scenario_evaluator.run_senarios import run_scenarios
from hourly_simulation.predict_demand import predict_demand_in_year
from hourly_simulation.parameters import simulation_params, NORMALISED_SOLAR_PRODUCTION
from tests.sanity_checks import test_simulation
import logging

logging.getLogger().setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
num_batteries = 1
print("*Under production*")
future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [9] * 24}))
price: CostElectricityDf = CostElectricityDf(pd.DataFrame({"Cost ILS/Kwh": [100 for i in range(80, 104)],
                                                           "HourOfYear": [i for i in range(80, 104)]}))
binary_price: CostElectricityDf = CostElectricityDf(pd.DataFrame({"Cost ILS/Kwh": [0 for i in range(80, 104)],
                                                           "HourOfYear": [i for i in range(80, 104)]}))

electricity_use: ElectricityUseDf = strategies.first_selling_strategy(
    demand=future_demand, production=total_panel_production,
    battery_capacity=num_batteries * simulation_params.BATTERY_CAPACITY,
    battery_power=num_batteries * simulation_params.CHARGE_POWER, battery_efficiency=0.87, cost_profile=price, binary_cost_profile=binary_price, sell_profile=price, sale_max_power=0.1)
# test_simulation(electricity_use=electricity_use, demand=future_demand, production=total_panel_production,
#                 params=simulation_params, num_batteries=num_batteries)
daily_graph(electricity_use.df)

print("*Over production*")
future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [11] * 24}))
electricity_use: ElectricityUseDf = strategies.first_selling_strategy(
    demand=future_demand, production=total_panel_production,
    battery_capacity=num_batteries * simulation_params.BATTERY_CAPACITY,
    battery_power=num_batteries * simulation_params.CHARGE_POWER, battery_efficiency=0.87, cost_profile=price, binary_cost_profile=binary_price, sell_profile=price, sale_max_power=0.1)
# test_simulation(electricity_use=electricity_use, demand=future_demand, production=total_panel_production,
#                 params=simulation_params, num_batteries=num_batteries)
daily_graph(electricity_use.df)
#
print("*Oscillations around demand*")
future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [10 + 2 * np.sin(2 * np.pi * x / 6)
                                                                                      for x in range(24)]}))
electricity_use: ElectricityUseDf = strategies.first_selling_strategy(
    demand=future_demand, production=total_panel_production,
    battery_capacity=num_batteries * simulation_params.BATTERY_CAPACITY,
    battery_power=num_batteries * simulation_params.CHARGE_POWER, battery_efficiency=0.87, cost_profile=price, binary_cost_profile=binary_price, sell_profile=price, sale_max_power=0.1)
# test_simulation(electricity_use=electricity_use, demand=future_demand, production=total_panel_production,
#                 params=simulation_params, num_batteries=num_batteries)
daily_graph(electricity_use.df)

In [72]:
num_batteries = 1
print("*Under production*")
binary_cost_list = [0] * 17 + [1] * 7
cost_list = [10] * 17 + [20] * 7

future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [9] * 24}))
price: CostElectricityDf = CostElectricityDf(pd.DataFrame({"Cost ILS/Kwh": cost_list,
                                                           "HourOfYear": [i for i in range(80, 104)]}))
binary_price: CostElectricityDf = CostElectricityDf(pd.DataFrame({"Cost ILS/Kwh": binary_cost_list,
                                                           "HourOfYear": [i for i in range(80, 104)]}))

# electricity_use: ElectricityUseDf = strategies.first_selling_strategy(
#     demand=future_demand, production=total_panel_production,
#     battery_capacity=num_batteries * simulation_params.BATTERY_CAPACITY,
#     battery_power=num_batteries * simulation_params.CHARGE_POWER, battery_efficiency=0.87, cost_profile=price, binary_cost_profile=binary_price, sell_profile=price, sale_max_power=0.1)


print("*Oscillating production")
future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [10 + 2 * np.sin(2 * np.pi * x / 6)
                                                                                      for x in range(24)]}))
electricity_use: ElectricityUseDf = strategies.first_selling_strategy(
    demand=future_demand, production=total_panel_production,
    battery_capacity=num_batteries * simulation_params.BATTERY_CAPACITY,
    battery_power=4, battery_efficiency=1, cost_profile=price, binary_cost_profile=binary_price, sell_profile=price, sale_max_power=10)
daily_graph(electricity_use.df)

*Under production*
*Oscillating production
day 0
day 0 have expansive hours


In [21]:
num_batteries = 2.1
solar_panel_power_kw = 5437
demand = DemandDf(pd.read_csv(r'data/simulation_demand_input/consumption_hatzor.csv', index_col=0))
future_demand = predict_demand_in_year(demand, params=simulation_params, simulated_year=2020)
total_panel_production = ProductionDf(NORMALISED_SOLAR_PRODUCTION.df.copy())
total_panel_production.df[total_panel_production.SolarProduction] *= solar_panel_power_kw
selling_cost = CostElectricityDf(pd.read_csv(r'data/electricity_cost.csv', index_col=0))
binary_cost = CostElectricityDf(pd.read_csv(r'data/electricity_cost_binary.csv', index_col=0))
electricity_use: ElectricityUseDf = strategies.first_selling_strategy(
    demand=future_demand, production=total_panel_production,
    battery_capacity=num_batteries * simulation_params.BATTERY_CAPACITY,
    battery_power=num_batteries * simulation_params.CHARGE_POWER, battery_efficiency=0.87, cost_profile=selling_cost, binary_cost_profile=binary_cost, sell_profile=selling_cost, sale_max_power=14000)
# test_simulation(electricity_use=electricity_use, demand=future_demand, production=total_panel_production,
#                 params=simulation_params, num_batteries=num_batteries)
yearly_graph(electricity_use.df, batteries_num=num_batteries, batteries_cap=num_batteries * simulation_params.BATTERY_CAPACITY)

AttributeError: 'int' object has no attribute 'df'